In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import IPython.display as ipd
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
SAMPLING_RATE=16000
DURATION=10
N_FFT = 1024
WIN_LENGTH = 1024
HOP_LENGTH = 512

In [ ]:
X = np.load('/content/drive/MyDrive/EE603_Project/combined_spectrogram.npy')
print(X.shape)
y = np.load('/content/drive/MyDrive/EE603_Project/y.npy')
print(y.shape)

(717, 513, 313)
(717,)


In [ ]:
def reshape(X, y):
  X_r = []
  y_r = []
  l = 0
  m = 0
  for i in range(len(X)):
    cur = X[i]
    l = cur.shape[1]
    m = cur.shape[0]
    cur_trans = np.transpose(cur)
    X_r.append(cur_trans)
    y_r.append([y[i]]*l)
  
  X_r = np.array(X_r)
  y_r = np.array(y_r)
  # print(X_r.shape)
  # print(y_r.shape)
  X_r = np.reshape(X_r, (-1,m))
  y_r = np.reshape(y_r, (-1,1))
  # print(X_r.shape)
  # print(y_r.shape)
  return X_r, y_r

In [ ]:
X, y = reshape(X, y)
print(X.shape)
print(y.shape)

(224421, 513)
(224421, 1)


In [ ]:
def one_hot_enc(cl_name):
  if(cl_name == 'music'):
    return np.array([0,1,0])
  elif(cl_name == 'speech'):
    return np.array([1,0,0])
  else:
    return np.array([0,0,1])

In [ ]:
def string2ohv(y):
  y_ohv = []
  for i in y:
    y_ohv.append(one_hot_enc(i))
  return np.array(y_ohv)

In [ ]:
y = string2ohv(y)
print(y.shape)

(224421, 3)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, TimeDistributed, LSTM

In [ ]:
def get_DNN_model():
  model = Sequential()
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.25))
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.25))
  model.add(Dense(16, activation='relu'))
  model.add(Dense(3, activation='softmax'))
  # model.summary()
  model.compile(loss='categorical_crossentropy', 
                optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
# input_shape = (X_train.shape[1], 1)
model = get_DNN_model()
model.fit(X_train, y_train, epochs=5, shuffle=True)

Epoch 1/5
5962/5962 [==============================] - 29s 5ms/step - loss: 0.6535 - accuracy: 0.6309
Epoch 2/5
5962/5962 [==============================] - 29s 5ms/step - loss: 0.4551 - accuracy: 0.7325
Epoch 3/5
5962/5962 [==============================] - 28s 5ms/step - loss: 0.3746 - accuracy: 0.8079
Epoch 4/5
5962/5962 [==============================] - 28s 5ms/step - loss: 0.3453 - accuracy: 0.8202
Epoch 5/5
5962/5962 [==============================] - 28s 5ms/step - loss: 0.3415 - accuracy: 0.8213


In [ ]:
model.save('/content/drive/MyDrive/models/DNN2')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/models/DNN2/assets


In [ ]:
from tensorflow import keras

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/models/DNN2')

In [ ]:
predictions = model.predict(X_test)

In [ ]:
print(predictions.shape)
print(y_test.shape)

(33664, 3)
(33664, 3)


In [ ]:
model.evaluate(X_test, y_test)

1052/1052 [==============================] - 3s 3ms/step - loss: 0.3030 - accuracy: 0.8655


[0.30296972393989563, 0.8654943108558655]

In [ ]:
for i in range(len(predictions)):
  for j in range(len(predictions[i])):
    if(predictions[i][j] > 0.5):
      predictions[i][j] = 1
    else:
      predictions[i][j] = 0